In [ ]:
'''

        
#Calculate percentage of each amino acid in the cell
#1- Sum normalized ratios
norm_sum = 0.
for letter in alphabet:
    value  = normalized_dict.get(letter)
    norm_sum  = value + norm_sum
    
print(norm_sum)

#2- Divide letter to norm_sum to get ratio of amino acid in the cell
ratio_dict = {'A':0.,'C':0.,'D':0.,'E':0.,'F':0.,'G':0.,'H':0.,'I':0.,
              'K':0.,'L':0.,'M':0.,'N':0.,'P':0.,'Q':0.,'R':0.,'S':0.,'T':0.,'V':0.,'W':0.,'Y':0.}

#Constant for the amount of protein in the cell
PROT_G = 5.9
for letter in alphabet:
    value = normalized_dict.get(letter)
    ratio  = value/norm_sum
    #Convert ratios to grams
    converted_ratio = ratio*PROT_G    
    ratio_dict[letter] = converted_ratio
    
    
#3- Convert gram ratios to mmol/g Dry weight

To verify that the normalized to grams to get to the total amount of protein 
(here everything is converted to grams instead of femto grams)
new_value = 0.
for letter in alphabet:
    value = ratio_dict.get(letter)
    new_value = new_value+value 
    
print(new_value)

letter_to_bigg = {'A':Mflorum.metabolites.ala__L_c,'C':Mflorum.metabolites.cys__L_c,'D':Mflorum.metabolites.asp__L_c,'E':Mflorum.metabolites.glu__L_c,'F':Mflorum.metabolites.phe__L_c,
                  'G':Mflorum.metabolites.gly_c,'H':Mflorum.metabolites.his__L_c,'I':Mflorum.metabolites.ile__L_c,'K':Mflorum.metabolites.lys__L_c,'L':Mflorum.metabolites.leu__L_c,
                  'M':Mflorum.metabolites.met__L_c,'N':Mflorum.metabolites.asn__L_c,'P':Mflorum.metabolites.pro__L_c,'Q':Mflorum.metabolites.gln__L_c,'R':Mflorum.metabolites.arg__L_c,
                  'S':Mflorum.metabolites.ser__L_c,'T':Mflorum.metabolites.thr__L_c,'V':Mflorum.metabolites.val__L_c,'W':Mflorum.metabolites.trp__L_c,'Y':Mflorum.metabolites.tyr__L_c}
CELL_WEIGHT = 9.7
biomass_ratios = {'A':0.,'C':0.,'D':0.,'E':0.,'F':0.,'G':0.,'H':0.,'I':0.,
                  'K':0.,'L':0.,'M':0.,'N':0.,'P':0.,'Q':0.,'R':0.,'S':0.,'T':0.,'V':0.,'W':0.,'Y':0.}
#Get number of moles from number of grams
for letter in alphabet:
    metab = letter_to_bigg.get(letter)
    mol_weight = metab.formula_weight
    grams = ratio_dict.get(letter)
    mmols_per_cell = (grams/mol_weight)*1000
    mmols_per_gDW = mmols_per_cell/CELL_WEIGHT
    biomass_ratios[letter] = mmols_per_gDW
    


'''

In [ ]:
'''
Update for protein
#Change the ratios based on their appearance in the reactants and products of biomass eq
'''
Algorythm should go like this:
- Find metabolites from BIOMASS (stack them in a list with their coefficients)
- Remove the metabolites from the BIOMASS equation
- Add them with the new coefficients
'''
#1- Find metabolites in BIOMASS
reactants_biomass, biomass_coefficients, reactant_removal =[],[],[]
for letter in alphabet:
    metab = letter_to_bigg.get(letter)
    coefficient = biomass_ratios.get(letter)
    for r in Mflorum.reactions.BIOMASS.reactants:
        if metab.id[0:3] in r.id:
            reactants_biomass.append(r)
            neg_coefficient = -coefficient
            biomass_coefficients.append(neg_coefficient)
            reactant_removal.append(1)

new_reactants_coefficients = dict(zip(reactants_biomass,biomass_coefficients))
remove_reactants = dict(zip(reactants_biomass,reactant_removal))

products_biomass, biomass_coefficients, product_removal =[],[],[]
for letter in alphabet:
    metab = letter_to_bigg.get(letter)
    coefficient = biomass_ratios.get(letter)
    for r in Mflorum.reactions.BIOMASS.products:
        if metab.id[0:3] in r.id:
            products_biomass.append(r)
            biomass_coefficients.append(coefficient)
            product_removal.append(-1)
            
new_products_coefficients = dict(zip(amino_acids_biomass,biomass_coefficients))
remove_products = dict(zip(products_biomass,product_removal))
    
remove_reactants

#2- Remove metabolites from BIOMASS equation
Mflorum.reactions.BIOMASS.add_metabolites(remove_reactants)
print(Mflorum.reactions.BIOMASS.reactants)

#3- Add reactants and products to biomass with their calculated ratios
Mflorum.reactions.BIOMASS.add_metabolites(new_reactants_coefficients)
print(Mflorum.reactions.BIOMASS.reactants)

#3- Add reactants and products to biomass with their calculated ratios
Mflorum.reactions.BIOMASS.add_metabolites(new_products_coefficients)
print(Mflorum.reactions.BIOMASS.reaction)

'''

In [7]:
#Generate an example input file from proteomics data
#The format is 2 columns, gene_id:abundance (could be csv, dataframe or dictionary)

#Obtain the total amount of protein from proteomic data
proteomics = pd.read_table('/home/jean-christophe/Documents/Maitrise_UCSD/Experimental validation/Proteomics/Genes_in_model_expression_list_summary.txt')


In [69]:
class Utilitaries():
    
    def chopStr(text):
        return (text[2:-2])

In [35]:
text = '01234'
Utilitaries.chopStr(text)

2


In [111]:
class Protein(Utilitaries,Update):
    import pandas as pd
    
    #Objects
    #none so far
    #Methods
    def get_coefficients(path_to_genbank,path_to_model,proteomics,CELL_WEIGHT=0.3, PROTEIN_RATIO=0.6):
        
        AMINO_ACIDS = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
        
        def import_genome(genbank):
            from Bio import SeqIO
            return SeqIO.read(genbank,'genbank')
        
        def import_model(path_to_model):
            import cobra
            extension = path_to_model.split('.')[-1]
            if extension == 'json':
                return cobra.io.load_json_model(path_to_model)
            elif extension == 'xml':
                return cobra.io.read_sbml_model(path_to_model)
            
        def get_protein_sequence(genome_record):
            #Get the sequence of each protein, locus tag and prot id from genbank file
            seq_list,protID_list,locus_list = [],[],[]
            for element in genome_record.features:
                if element.type == 'CDS':
                    locus_list.append(element.qualifiers['locus_tag'][0])
                    protID_list.append(element.qualifiers['protein_id'][0])
                    seq_list.append(element.qualifiers['translation'][0])

            return pd.DataFrame({'Locus_tag':locus_list, 'Protein_ID':protID_list, 'Sequence':seq_list})
        
        def get_aa_composition(df1):
        #For each protein find the amino acid composition
        #Outputs a dictionnary of dictionnaries where:
        #Keys = locus_tag
        #Values = A dictionary for each amino acid
            #This dictionary contains:
            #Keys = amino acid by letter code
            #Values = the occurence of that amino acid
            list_of_dict =[]
            for i, row in df1.iterrows():
                list_of_occurences = []
                #Get the occurence for each letter
                for letter in AMINO_ACIDS:
                    protein_sequence = row.Sequence
                    occurence_of_letter = protein_sequence.count(letter)
                    list_of_occurences.append(occurence_of_letter)
                #Generate dictionary of occurences for a given gene
                dict_of_occurences = dict(zip(AMINO_ACIDS,list_of_occurences))
                #Generate dict for each gene
                dict_per_locus = {row.Locus_tag:dict_of_occurences}
                #Store the amount of each amino acid per gene in a list
                list_of_dict.append(dict_per_locus)
                
            return list_of_dict

        def normalize_aa_composition(list_of_dict):
            #Normalize the value of each amino acid per protein following proteomic data
            normalized_dict = {'A':0.,'C':0.,'D':0.,'E':0.,'F':0.,'G':0.,'H':0.,'I':0.,
                               'K':0.,'L':0.,'M':0.,'N':0.,'P':0.,'Q':0.,'R':0.,'S':0.,'T':0.,'V':0.,'W':0.,'Y':0.}
            for d in list_of_dict:
                #Get the coefficient from proteomics
                text = str(list(d.keys())[0])
                coeff = coeff_dict.get(text)
                #If no protein abundance coefficient is 0.
                try:
                    coeff_number = float(coeff)
                except:
                    coeff_number = 0.
                    
                #Multiply each amino acid by the coefficient
                amino_acids = list(d.values())
                for letter in AMINO_ACIDS:
                    value = float(amino_acids[0].get(letter))
                    #Update the normalized value
                    normalized_value = value*coeff_number
                    new_value = normalized_dict.get(letter) + normalized_value
                    normalized_dict[letter] = new_value
                    
            return normalized_dict
         
        #Operations
        #1- Parse the genome, extract protein sequence, count and store amino acid composition of each protein
        genome_record = import_genome(path_to_genbank)
        df1 = get_protein_sequence(genome_record)
        list_of_dict = get_aa_composition(df1)
        normalized_dict = normalize_aa_composition(list_of_dict)
        
        def get_norm_sum(normalized_dict):
            #1- Sum normalized ratios
            norm_sum = 0.
            for letter in AMINO_ACIDS:
                value  = normalized_dict.get(letter)
                norm_sum  = value + norm_sum

            return norm_sum

        def get_ratio(normalized_dict,norm_sum,PROTEIN_RATIO):
            #2- Divide letter to norm_sum to get ratio of each amino acid in the cell 
            # based on proteomic data
            ratio_dict = {'A':0.,'C':0.,'D':0.,'E':0.,'F':0.,'G':0.,'H':0.,'I':0.,
                          'K':0.,'L':0.,'M':0.,'N':0.,'P':0.,'Q':0.,'R':0.,'S':0.,'T':0.,'V':0.,'W':0.,'Y':0.}

            #Constant for the amount of protein in the cell
            PROTEIN_WEIGHT = CELL_WEIGHT*PROTEIN_RATIO
            for letter in AMINO_ACIDS:
                value = normalized_dict.get(letter)
                ratio  = value/norm_sum
                #Convert ratios to grams
                converted_ratio = ratio*PROTEIN_WEIGHT    
                ratio_dict[letter] = converted_ratio
                
            return ratio_dict
    
        def convert_to_coefficient(path_to_model,CELL_WEIGHT):
            model = import_model(path_to_model)
            #3- Convert gram ratios to mmol/g Dry weight
            '''
            To verify that the normalized to grams to get to the total amount of protein 
            (here everything is converted to grams instead of femto grams)
            '''
            letter_to_bigg = {'A':model.metabolites.ala__L_c,'C':model.metabolites.cys__L_c,'D':model.metabolites.asp__L_c,'E':model.metabolites.glu__L_c,'F':model.metabolites.phe__L_c,
                              'G':model.metabolites.gly_c,'H':model.metabolites.his__L_c,'I':model.metabolites.ile__L_c,'K':model.metabolites.lys__L_c,'L':model.metabolites.leu__L_c,
                              'M':model.metabolites.met__L_c,'N':model.metabolites.asn__L_c,'P':model.metabolites.pro__L_c,'Q':model.metabolites.gln__L_c,'R':model.metabolites.arg__L_c,
                              'S':model.metabolites.ser__L_c,'T':model.metabolites.thr__L_c,'V':model.metabolites.val__L_c,'W':model.metabolites.trp__L_c,'Y':model.metabolites.tyr__L_c}

            biomass_coefficients = {'A':0.,'C':0.,'D':0.,'E':0.,'F':0.,'G':0.,'H':0.,'I':0.,
                              'K':0.,'L':0.,'M':0.,'N':0.,'P':0.,'Q':0.,'R':0.,'S':0.,'T':0.,'V':0.,'W':0.,'Y':0.}
            #Get number of moles from number of grams
            for letter in AMINO_ACIDS:
                metab = letter_to_bigg.get(letter)
                mol_weight = metab.formula_weight
                grams = ratio_dict.get(letter)
                mmols_per_cell = (grams/mol_weight)*1000
                mmols_per_gDW = mmols_per_cell/CELL_WEIGHT
                biomass_coefficients[letter] = mmols_per_gDW
            
            return biomass_coefficients

        #2- Get coefficients from experimental proteomics data
        #Proteomics data should come in a 2 columns standard format gene_id:abundance
        norm_sum = get_norm_sum(normalized_dict)
        ratio_dict = get_ratio(normalized_dict,norm_sum,PROTEIN_RATIO)
        biomass_coefficients = convert_to_coefficient(path_to_model,CELL_WEIGHT)
        print(biomass_coefficients)

In [115]:
path_to_genbank = '/Users/JC/Documents/Maitrise_UCSD/GenBankFiles/Mflorum.gbff'
path_to_model = '/Users/JC/Documents/Maitrise_UCSD/Functionnal_testing/finalMeso1.json'
Protein.get_coefficients(path_to_genbank,path_to_model,proteomics,CELL_WEIGHT=0.3,PROTEIN_RATIO=0.6)

{'R': 0.12698594044292982, 'M': 0.10039334382981341, 'K': 0.4228955466570605, 'P': 0.14989571771940108, 'E': 0.32360384989600366, 'H': 0.05279661803935423, 'Q': 0.12769799109529054, 'V': 0.36217751226096007, 'Y': 0.09542866936675634, 'W': 0.01970009517084819, 'T': 0.2918719442117945, 'D': 0.24063727925750836, 'A': 0.5159110997956983, 'L': 0.3701278253832807, 'C': 0.024764164647747312, 'G': 0.514084868214509, 'F': 0.13775204382857809, 'N': 0.26425478270623226, 'I': 0.38866814279916834, 'S': 0.32422059315282087}


In [10]:
import os

In [11]:
os.getcwd()

'/Users/JC/Documents/Maitrise_UCSD/Functionnal_testing/biomass'

In [1]:
#Generate proteomics file to be manipulated by package
import pandas as pd

uniprot_conversion = pd.read_table('/Users/JC/Documents/Maitrise_UCSD/Experimental_validation/Proteomics/Uniprot_conversion.csv')
peptide_shaker = pd.read_csv('/Users/JC/Documents/Maitrise_UCSD/Experimental_validation/Proteomics/20160723_2D_SRodrigue_ALL/reports/20160723_2D_SRodrigue_ALL_sample_1_Default_Hierarchical_Report.txt')

IOError: File /Users/JC/Documents/Maitrise_UCSD/Experimental_validation/Proteomics/Uniprot_conversion.csv does not exist

In [17]:
#Normalize by the abundance of protein given by proteomic data
coefficients, accessions = [], []
for i,row in peptide_shaker.iterrows():
    n = str(row['Unnamed: 0'])
    if '.' not in n and n != 'nan':
        coefficients.append(row['Spectrum Counting NSAF'])
        accessions.append(row['Main Accession'])  
        
coeff_accession = pd.DataFrame({'NSAF': coefficients, 'Accession': accessions})

In [20]:
accessions2, mfl_ids = [],[]
for i, row in uniprot_conversion.iterrows():
    accessions2.append(row['yourlist:M20170121A7434721E10EE6586998A056CCD0537E674960X'])
    mfl_ids.append(row['Gene names'])

#Generate a conversion dict for protein accession numbers to locus tags
import re
new_mfl = []
for element in mfl_ids:
    new_mfl.append('Mfl'+ re.findall('(?<=Mfl).+', element)[0])
    
    
conv_dict = dict(zip(accessions2,new_mfl))

In [27]:
accession_mfl = []
for i, row in coeff_accession.iterrows():
    accession_mfl.append(conv_dict.get(row['Accession']))
    
coeff_dict = dict(zip(accession_mfl, coefficients))
proteomics = pd.DataFrame({'gene_id':[k for k in coeff_dict.keys()], 
                           'abundance':[v for v in coeff_dict.values()]}
                          ,columns=['gene_id','abundance'])

,gene_id,abundance
0,Mfl172,0.0716267146123262
1,Mfl392,0.208204555050382
2,Mfl052,0.0168350168350168
3,Mfl306,0.132590145884944
4,Mfl675,0.0341816078658184
5,Mfl091,0.0772421755594832
6,Mfl655,0.0523598820058997
7,Mfl201,2.0660815173527
8,Mfl613,0.0168300367164707
9,Mfl341,0.115384615384615
